In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%%capture
!pip install mlflow pyngrok

In [ ]:
import mlflow

# Set up MLflow experiment (optional, if not already set)
mlflow.set_tracking_uri("file:///kaggle/working/mlruns")  # Saves runs in /kaggle/working/mlruns
mlflow.set_experiment("Classification_Finetuining_Mistral_Model_Experiment")
mlflow.start_run(run_name="Mistral")

2024/11/16 14:34:10 INFO mlflow.tracking.fluent: Experiment with name 'Classification_Finetuining_Mistral_Model_Experiment' does not exist. Creating a new experiment.


<ActiveRun: >

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2ovuOjLYtKu4y3ljjztiABpxl41_4Kumw4mXHcWeDTWGwUk3L")

# Start MLflow UI
get_ipython().system_raw("mlflow ui --port 5000 &")
# Expose the MLflow UI on port 5000# Expose the MLflow UI on port 5000
public_url = ngrok.connect("5000", "http")
print(f"MLflow UI accessible at: {public_url}")

MLflow UI accessible at: NgrokTunnel: "https://c64c-34-23-62-139.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model_id="mistralai/Mistral-7B-Instruct-v0.2"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id, #"unsloth/Llama-3.2-3B-bnb-4bit",
    max_seq_length # Add LoRA adapters so we only need to update 1 to 10% of all parameters!
= max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[2024-11-16 14:34:14 +0000] [216] [INFO] Starting gunicorn 23.0.0
[2024-11-16 14:34:14 +0000] [216] [INFO] Listening at: http://127.0.0.1:5000 (216)
[2024-11-16 14:34:14 +0000] [216] [INFO] Using worker: sync
[2024-11-16 14:34:14 +0000] [217] [INFO] Booting worker with pid: 217
[2024-11-16 14:34:14 +0000] [218] [INFO] Booting worker with pid: 218
[2024-11-16 14:34:14 +0000] [219] [INFO] Booting worker with pid: 219
[2024-11-16 14:34:14 +0000] [220] [INFO] Booting worker with pid: 220


==((====))==  Unsloth 2024.11.7: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

# Add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
# Create a dictionary of parameters to log
model_params = {
    "Mistral_model_name": "mistralai/Mistral-7B-Instruct-v0.2",
    "Mistral_max_seq_length": max_seq_length,
    "Mistral_dtype": dtype,
    "Mistral_load_in_4bit": load_in_4bit,
}

# Log all parameters at once
mlflow.log_params(model_params)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","lm_head"],
    lora_alpha = 16,
    lora_dropout = 0.1, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.11.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Training lm_head in mixed precision to save VRAM


In [ ]:
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model_params = {
    "LoRA_r": 64,
    "LoRA_target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],
    "LoRA_lora_alpha": 16,
    "LoRA_lora_dropout": 0.1,
    "LoRA_bias": "none",
    "LoRA_use_gradient_checkpointing": True,
    "LoRA_random_state": 3407,
    "LoRA_use_rslora": False,
    "LoRA_loftq_config": None,
}

# Log model parameters
mlflow.log_params(model_params)

# Define tokenizer settings for logging
tokenizer_settings = {
    "tokenizer_padding_side": "right",
    "tokenizer_add_eos_token": True,
    "tokenizerpad_token": tokenizer.eos_token,
}

# Log tokenizer settings
mlflow.log_params(tokenizer_settings)

# Data Preparation

In [ ]:
from datasets import load_dataset
import pandas as pd
import mlflow

# Load the dataset
dataset = load_dataset('csv', data_files='/kaggle/input/train-dataset/Classification_train.csv', split='train')

# Log dataset parameters with MLflow
mlflow.log_param("dataset_name", "Classification_train.csv")
mlflow.log_param("num_rows", dataset.num_rows)
mlflow.log_param("num_columns", len(dataset.column_names))
mlflow.log_param("column_names", dataset.column_names)

# Convert to pandas DataFrame for manipulation
classification_train = dataset.to_pandas()

# Take a 10% sample for the test dataset
classification_test = classification_train.sample(frac=0.1, random_state=42)

# Drop the sampled rows from the original dataset to get the remaining 90%
classification_train = classification_train.drop(classification_test.index)

# Save the test and updated train datasets
classification_test.to_csv('classification_test.csv', index=False)
classification_train.to_csv('classification_train.csv', index=False)

print("10% sample saved as classification_test.csv and removed from classification_train.csv")

# Log a sample of the dataset as an artifact
sample_df = dataset.to_pandas().sample(5)  # Taking a small sample for logging
sample_df.to_csv("dataset_sample.csv", index=False)
mlflow.log_artifact("dataset_sample.csv", artifact_path="dataset_sample")


Generating train split: 0 examples [00:00, ? examples/s]

10% sample saved as classification_test.csv and removed from classification_train.csv


In [ ]:
classification_train = pd.read_csv('/kaggle/input/train-dataset/Classification_train.csv')

# Take a 10% sample for the test dataset
classification_test = classification_train.sample(frac=0.1, random_state=42)

# Drop the sampled rows from the original dataset to get the remaining 90%
classification_train = classification_train.drop(classification_test.index)

# Save the test and updated train datasets
classification_test.to_csv('classification_test.csv', index=False)
classification_train.to_csv('classification_train.csv', index=False)

print("10% sample saved as classification_test.csv and removed from classification_train.csv")

10% sample saved as classification_test.csv and removed from classification_train.csv


In [ ]:
dataset = load_dataset('csv', data_files='/kaggle/working/classification_train.csv', split='train')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['dataset_name', 'series_description', 'algorithm', 'hyperparameters'],
    num_rows: 727
})

In [ ]:
train_valid_split = dataset.train_test_split(test_size=0.1)

In [ ]:
train_dataset = train_valid_split['train']
valid_dataset = train_valid_split['test']

In [ ]:
EOS_TOKEN = tokenizer.eos_token

train_prompt = """Below is a description for a time series data. Write a response that gives the name of the best fitting machine learning algorithm in one word without explanation.
The best algorithm name should be one of this search space algorithms: AdaboostRegressor, ElasticNetRegressor,  ExtraTreesRegressor,  LassoRegressor,  LightgbmRegressor, SVR, GaussianProcessRegressor, RandomForestRegressor or  XGBoostRegressor.

### DESCRIPTION:
{}

### RESPONSE:
{}"""



def formatting_prompts_func(examples):
    inputs       = examples["series_description"]
    outputs      = examples["algorithm"]
    texts = []
    for input, output in zip( inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = train_prompt.format( input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


def formatting_test_prompts_func(examples):
    inputs = examples["series_description"]
    texts = []

    for input in inputs:
        text = test_prompt.format(input)
        texts.append(text)

    return { "text": texts }

In [ ]:
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/654 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['dataset_name', 'series_description', 'algorithm', 'hyperparameters', 'text'],
    num_rows: 654
})

In [ ]:
valid_dataset = valid_dataset.map(formatting_prompts_func, batched = True)
valid_dataset

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Dataset({
    features: ['dataset_name', 'series_description', 'algorithm', 'hyperparameters', 'text'],
    num_rows: 73
})

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_arguments= TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 32//4,
        gradient_checkpointing=True,
        warmup_steps = 5,
        max_steps = -1, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "./mistral_outputs",
        evaluation_strategy="steps", #epoch
        save_strategy="epoch",
    )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from trl import  DataCollatorForCompletionOnlyLM

instruction_template="DESCRIPTION:"
response_template = "RESPONSE:"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_arguments,
    # data_collator =  DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,
    #                                                  response_template=response_template,
    #                                                  tokenizer=tokenizer,mlm=False),

)

Map (num_proc=2):   0%|          | 0/654 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/73 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.715 GB of memory reserved.


In [ ]:
import time
start= time.time()
trainer_stats = trainer.train()
print((time.time()-start)/60)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 654 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 20
 "-____-"     Number of trainable parameters = 298,844,160
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
1,2.235300,2.229566
2,2.248900,1.811847
3,1.816900,1.269612
4,1.272800,0.876614
5,0.870700,0.624159
6,0.641400,0.468591
7,0.469700,0.354218
8,0.352700,0.294866
9,0.298800,0.263922
10,0.283200,0.246096


51.465876722335814


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3084.6466 seconds used for training.
51.41 minutes used for training.
Peak reserved memory = 10.781 GB.
Peak reserved memory for training = 5.066 GB.
Peak reserved memory % of max memory = 73.136 %.
Peak reserved memory for training % of max memory = 34.367 %.


In [ ]:
mlflow.end_run()

In [ ]:
!zip -r mlruns.zip /kaggle/working/mlruns

updating: kaggle/working/mlruns/ (stored 0%)
updating: kaggle/working/mlruns/446966349057200023/ (stored 0%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/ (stored 0%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/metrics/ (stored 0%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/metrics/eval_steps_per_second (deflated 66%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/metrics/train_samples_per_second (stored 0%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/metrics/loss (deflated 58%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/metrics/train_loss (stored 0%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0/metrics/learning_rate (deflated 67%)
updating: kaggle/working/mlruns/446966349057200023/68e2d839c3a54258a9c5fd35dbb3b2f0